In [21]:
path_nfl_qb_stats = '../../capstone_project_1/nfl-qb-stats'
path_nfl_passing = '../../capstone_project_1/nfl-passing-statistics'

In [22]:
%matplotlib inline
import pandas as pd
import statistics
import numpy as np
import os
import glob

In [23]:
qb_game_stats = pd.read_csv(path_nfl_qb_stats+'/QBStats_all.csv')
passing_stats = pd.concat(map(pd.read_csv, glob.glob(path_nfl_passing+ '/pass*.csv')),sort = True)


In [24]:
#load csv of 2019 stats into dataframe, columns in different order than 2009-2018
passing_stats_2019 = pd.read_csv(path_nfl_passing + '/unordered-pass-2019.csv')
passing_stats_2019 = passing_stats_2019.rename(columns={'Qbrec':'QBrec'})

In [25]:
#concatenate 2009-2018 w/ 2019
passing_stats=pd.concat([passing_stats, passing_stats_2019], sort=True, ignore_index=True)
passing_stats = passing_stats.drop(['1D', 'Yds.1'], axis = 1)

In [26]:
passing = passing_stats[((passing_stats['Pos'] == 'QB') | (passing_stats['Pos'] == 'qb')) & (passing_stats['Att'] >= 100)]

In [27]:
#avg NFL YPA of QBs with more than 100 att from 2009-2019
ypa_avg = round(passing['Y/A'].mean(), 1)
ypa_avg

7.1

In [28]:
#avg NFL ANY/A of QBs with more than 100 att from 2009-2019
anya_avg = round(passing['ANY/A'].mean(),1)
anya_avg

5.9

In [29]:
#split index into player names only
qb_stats = passing.set_index(['Player']).sort_index()
qb_stats.index = qb_stats.index.str.split("+" , n=1, expand = True)
qb_stats = qb_stats.droplevel(1)

In [30]:
qb_stats.index = qb_stats.index.str.split('*', n=1, expand = True)
qb_stats=qb_stats.droplevel(1)

In [31]:
qb_stats.index = qb_stats.index.str.split('\\', n=1, expand = True)

In [32]:
qb_stats = qb_stats.droplevel(1)

In [33]:
#calculate career win percentage
qb_record = qb_stats['QBrec']
qb_record = qb_record.iloc[:].str.split(r'\-|/', n = 1, expand = True)
qb_record[1] = qb_record.iloc[:,1].str.split(r'\-|/', n=1, expand = True)
qb_record = qb_record.rename(columns={0: 'win', 1:'loss'})
qb_record[['win', 'loss']] = qb_record[['win','loss']].astype(int)
qb_record = qb_record.groupby(level=0).sum()
qb_record['win_percentage'] = (qb_record['win'] / (qb_record['win'] + qb_record['loss']))

In [34]:
#qb total int 
qb_int = qb_stats['Int']
qb_int = qb_int.groupby(qb_int.index).sum()

#qb total td
qb_td = qb_stats['TD']
qb_td = qb_td.groupby(qb_td.index).sum()

#qb total yards
qb_yards = qb_stats['Yds']
qb_yards = qb_yards.groupby(qb_yards.index).sum()

#qb total games
qb_games = qb_stats['G']
qb_games = qb_games.groupby(qb_games.index).sum()

#qb sacks
qb_sacks = qb_stats['Sk']
qb_sacks = qb_sacks.groupby(qb_sacks.index).sum()

In [35]:
#qb average for qbs playing from 2009-2019, replace necessary columns with career totals 
qb_career_stats = qb_stats.groupby(level=0).mean().round(1)
qb_career_stats.sort_values(by=['Y/A'], ascending=False)
qb_career_stats['win_pct'] = qb_record['win_percentage']
qb_career_stats['Int'] = qb_int
qb_career_stats['TD'] = qb_td
qb_career_stats['Yds'] = qb_yards
qb_career_stats['G'] = qb_games
qb_career_stats['Sk'] = qb_sacks

In [36]:
qb_career_stats = qb_career_stats.drop(['4QC','Age','GWD','GS','Rk', 'Lng'], axis = 1)

In [37]:
qb_career_stats.info()
#one missing value in DataFrame(QBR column), will fill with average of columnb

<class 'pandas.core.frame.DataFrame'>
Index: 132 entries, A.J. McCarron to Zach Mettenberger
Data columns (total 20 columns):
ANY/A      132 non-null float64
AY/A       132 non-null float64
Att        132 non-null float64
Cmp        132 non-null float64
Cmp%       132 non-null float64
G          132 non-null int64
Int        132 non-null int64
Int%       132 non-null float64
NY/A       132 non-null float64
QBR        131 non-null float64
Rate       132 non-null float64
Sk         132 non-null int64
Sk%        132 non-null float64
TD         132 non-null int64
TD%        132 non-null float64
Y/A        132 non-null float64
Y/C        132 non-null float64
Y/G        132 non-null float64
Yds        132 non-null int64
win_pct    132 non-null float64
dtypes: float64(15), int64(5)
memory usage: 21.7+ KB


In [54]:
qb_career_stats

,ANY/A,AY/A,Att,Cmp,Cmp%,G,Int,Int%,NY/A,QBR,Rate,Sk,Sk%,TD,TD%,Y/A,Y/C,Y/G,Yds,win_pct
A.J. McCarron,6.3,7.4,119.0,79.0,66.4,7,2,1.7,6.0,NaN,97.1,12,9.2,6,5.0,7.2,10.8,122.0,854,0.666667
Aaron Rodgers,7.4,8.4,496.9,321.5,64.9,158,70,1.4,6.8,67.6,103.6,405,7.0,335,6.2,7.8,12.0,268.6,42579,0.681529
Alex Smith,6.2,7.1,414.1,264.7,64.0,134,70,1.8,6.2,54.3,91.8,329,7.4,174,4.3,7.1,11.1,217.7,29389,0.638462
Andrew Luck,6.3,7.0,548.3,333.3,60.4,86,83,2.6,6.4,63.1,88.3,174,5.0,171,5.2,7.1,11.9,274.7,23671,0.616279
Andy Dalton,6.1,6.9,510.5,316.4,62.1,122,107,2.6,6.4,50.2,87.9,257,5.9,183,4.5,7.1,11.5,239.0,29028,0.541667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Troy Smith,6.0,7.6,145.0,73.0,50.3,6,4,2.8,6.5,38.5,77.8,18,11.0,5,3.4,8.1,16.1,196.0,1176,0.500000
Tyler Palko,3.0,3.9,134.0,80.0,59.7,6,7,5.2,4.9,29.6,59.8,11,7.6,2,1.5,5.9,10.0,132.7,796,0.250000
Tyrod Taylor,5.9,7.0,428.0,266.0,62.2,30,10,1.2,5.7,60.7,89.4,88,9.4,31,3.6,6.8,10.9,194.0,5822,0.517241
Vince Young,6.0,6.7,176.3,103.7,58.7,27,19,4.2,6.9,57.1,80.7,30,5.9,24,4.6,7.6,13.0,146.8,4000,0.619048


In [38]:
qb_career_stats['QBR'].fillna(qb_career_stats['QBR'].mean())

A.J. McCarron        46.534351
Aaron Rodgers        67.600000
Alex Smith           54.300000
Andrew Luck          63.100000
Andy Dalton          50.200000
                       ...    
Troy Smith           38.500000
Tyler Palko          29.600000
Tyrod Taylor         60.700000
Vince Young          57.100000
Zach Mettenberger    24.600000
Name: QBR, Length: 132, dtype: float64

In [39]:
qb_career_stats.to_csv('nfl-passing.csv')

In [40]:
#plot ypa vs winpct, qbs with more than 100 career attempts
#ypa_vs_winpct = qb_career_stats[['Y/A', 'win_pct']]
#ypa_vs_winpct.plot(x='win_pct', y='Y/A', kind='scatter')

In [41]:
#correlation
#ypa_vs_winpct.corr(method='pearson')

In [42]:
#anya vs winpct, qbs with more than 100 attempts
#anya_vs_winpct = qb_career_stats[['ANY/A', 'win_pct']]
#anya_vs_winpct.plot(x='win_pct', y = 'ANY/A', kind='scatter')

In [43]:
#correlation
#anya_vs_winpct.corr(method='pearson')

In [44]:
#ypa vs winpct, qbs with more than 200 attempts
#ypa_200 = qb_career_stats[qb_career_stats['Att'] >= 200]
#ypa_200 = ypa_200[['Y/A', 'win_pct']]
#ypa_200.plot(x='win_pct',y='Y/A', kind = 'scatter')

In [45]:
#correlation
#ypa_200.corr(method='pearson')

In [46]:
#QBR vs win_pct
#qbr_winpct = qb_career_stats[['QBR', 'win_pct']]
#qbr_winpct.plot(x='win_pct', y='QBR', kind = 'scatter')

In [47]:
#qbr_winpct.corr(method='pearson')

In [48]:
#qbs with ypa > league average during period
#qbs_ypa = qb_avg_stats[qb_avg_stats['Y/A'] > ypa_avg].sort_values(by=['Y/A'], ascending=False)
#qbs_ypa.style.background_gradient('Blues',axis = 1, subset = ['Y/A'])

In [49]:
#qbs_anya = qb_stats.groupby(level=0).mean().round(1)
#qbs_anya  = qbs_anya.sort_values(by=['ANY/A'], ascending = False)
#qbs_anya